In [1]:
# https://www.youtube.com/watch?v=PQCE9ChuIDY&list=PLeo1K3hjS3uu7CxAacxVndI4bE_o3BDtO&index=13

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline

In [3]:
df = pd.read_csv("insurance_data.csv")
df.head()

,age,affordibility,bought_insurance
0,22,1,0
1,25,0,0
2,47,1,1
3,52,0,0
4,46,1,1


In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df[["age","affordibility"]], df.bought_insurance, test_size = 0.2, random_state = 42)

In [6]:
X_train_scaled = X_train.copy()
X_train_scaled["age"] = X_train_scaled["age"] / 100
X_train_scaled

,age,affordibility
17,0.58,1
22,0.40,1
11,0.28,1
13,0.29,0
15,0.55,1
1,0.25,0
4,0.46,1
5,0.56,1
2,0.47,1
16,0.25,0


In [7]:
X_test_scaled = X_test.copy()
X_test_scaled["age"] = X_test_scaled["age"] / 100
X_test_scaled

,age,affordibility
9,0.61,1
25,0.54,1
8,0.62,1
21,0.26,0
0,0.22,1
12,0.27,0


In [19]:
def sigmoid_numpy(X):
   return 1/(1+np.exp(-X))

# sigmoid_numpy(np.array([12,0,1]))

def log_loss(y_true, y_predicted):
    epsilon = 1e-15
    y_predicted_new = [max(i,epsilon) for i in y_predicted]
    y_predicted_new = [min(i,1-epsilon) for i in y_predicted_new]
    y_predicted_new = np.array(y_predicted_new)
    return -np.mean(y_true*np.log(y_predicted_new)+(1-y_true)*np.log(1-y_predicted_new))

In [25]:
class myNN:
    def __init__(self):
        self.w1 = 1
        self.w2 = 1
        self.bias = 0
    
    def fit(self, X, y, epochs, loss_thresold):
        self.w1, self.w2, self.bias = self.gradient_descent(X['age'],X['affordibility'],y, epochs, loss_thresold)
        
    def predict(self, X_test):
        weighted_sum = self.w1 * X_test['age'] + self.w2 * X_test['affordibility'] + self.bias
        return sigmoid_numpy(weighted_sum)
    
    def gradient_descent(self, age, affordibility, y_true, epochs, loss_thresold):
        # w1, w2, bias
        w1 = w2 = 1
        bias = 0
        rate = 0.5
        n = len(age)

        for i in range(epochs):
            weighted_sum = w1 * age + w2 * affordibility + bias
            y_predicted = sigmoid_numpy(weighted_sum)

            loss = log_loss(y_true, y_predicted)

            w1d = (1/n)*np.dot(np.transpose(age),(y_predicted - y_true))
            w2d = (1/n)*np.dot(np.transpose(affordibility),(y_predicted - y_true))

            bias_d = np.mean(y_predicted-y_true)

            w1 = w1 - rate * w1d
            w2 = w2 - rate * w2d
            bias = bias - rate * bias_d
            
            if i%50 == 0:
                print(f'Epoch: {i}, w1: {w1}, w1: {w2}, bias: {bias}, loss: {loss}')
            
            if loss <= loss_thresold:
                break

        return w1, w2, bias
        

In [26]:
customModel = myNN()
customModel.fit(X_train_scaled, y_train, epochs = 1000, loss_thresold = 0.4631)

Epoch: 0, w1: 0.9736899318847281, w1: 0.931388810977659, bias: -0.11748951666770448, loss: 0.7428288579142563
Epoch: 50, w1: 1.5242798722391315, w1: 0.8822187836689879, bias: -1.1310088596841463, loss: 0.5943161377198863
Epoch: 100, w1: 2.228187036337397, w1: 1.021207570480777, bias: -1.521934833689293, loss: 0.5674253359204205
Epoch: 150, w1: 2.8788802020633857, w1: 1.0918966282424585, bias: -1.8376731064036331, loss: 0.5462065237139112
Epoch: 200, w1: 3.4732864120499114, w1: 1.1354958313845407, bias: -2.1094943767610346, loss: 0.5289749850551891
Epoch: 250, w1: 4.0142352512851085, w1: 1.1683072115461934, bias: -2.3515205001257864, loss: 0.5148317335261617
Epoch: 300, w1: 4.506491403088422, w1: 1.19679207677037, bias: -2.57083460061376, loss: 0.5031414005690805
Epoch: 350, w1: 4.955154772161111, w1: 1.2234470550440062, bias: -2.77147562324279, loss: 0.49341774920289283
Epoch: 400, w1: 5.36504199155145, w1: 1.2491715720776853, bias: -2.956078966702516, loss: 0.48528128420193223
Epoch: 

In [28]:
customModel.predict(X_test_scaled)

9     0.866457
25    0.802014
8     0.874054
21    0.137819
0     0.319842
12    0.146012
dtype: float64